# Prosept. Создание сервиса для полуавтоматической разметки товаров


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sentence_transformers import SentenceTransformer

import re

import requests
import json

In [2]:
import sentence_transformers

In [3]:
#print(sentence_transformers.__version__)

In [4]:
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

## Введение

Цели данного этапа: разработка прототипа модели и скорейшая проработка важных вопросов, таких как общий вид интеграции в backend. После того, как данное решение пройдёт полный цикл от разбработки до проверки, команда DS приступит к решению дополнительных вопросов, вынесеных в список к проработке в дальнейшем и улучшению метрики. <br>
За основную гипотезу выбрано предположение о том, что названий товаров достаточно для получения мэтча с приемлемой точностью. Для реализации проверки этой гипотезы выбран метод `cosine_similarity`, применённый к векторам названий, полученным с помощью `TfidfVectorizer()` и `SentenceTransformer('LaBSE')`. Сопоставляются вектора столбцов `df_dealerprice.product_name` и `df_product.name`. Остальные фичи не используются, так по итогам проверки гипотезы получен удовлетворительный результат. <br>
Также сразу стоит отметить, что по договорённости с бэкендом наш модуль вызвается один раз при запуске программы, просчитывает все мэтчи и выдаёт таблицу с мэтчами, с которой в дальнейшем и работает бэкенд. Функция просчёта мэтчей (самая ресурсоёмкая операция) в тетрадке выполняется за **менее чем за 1** секунду, что говорит о том, что и в сборке выполнение данного кода не должно приводить к задержкам и неудобствам для оператора.

## Подготовка данных

Загрузим данные их датасетов `marketing_dealerprice.csv`, `marketing_product.csv` и `marketing_productdealerkey.csv`. `marketing_dealer` не нужен для DS модуля, так как информация из него подтягивается бэкендом. В контейнере реализована загрузка датасетов с помощью `requests.get()`.

Датасеты `marketing_dealerprice.csv` и `marketing_product.csv` сразу очистим от дубликатов, чтобы не повторять расчёт для одинаковых позиций. Стоит отметить, что в датасетах обнаружены одинаковые названия с разными `id`, а также пропуски. Данные вопросы вынесены в список к проработке в дальнейшем. <br>
Также для `df_product` создан список уникальных и упорядоченных `id_unique`. Данная операция осталась от предыдущего метода совмещения исходных данных, который за 19 часов до дэдлайна оказался нежизнеспособным. В дальнейшем будет рассмотрена возможность упразднения данной операции.

In [5]:
path_to_dir = '../data/'

In [6]:
df_dealerprice = pd.read_csv(path_to_dir + 'marketing_dealerprice.csv', sep=';')
df_dealerprice[['product_key', 'product_name']].nunique()

product_key     1965
product_name    1953
dtype: int64

In [7]:
df_dealerprice = df_dealerprice[['product_key', 'product_name']]
df_dealerprice.drop_duplicates(inplace=True)
df_dealerprice.head(2)

,product_key,product_name
0,546227,"Средство универсальное Prosept Universal Spray, 500мл"
1,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л"


In [8]:
df_product = pd.read_csv(path_to_dir + 'marketing_product.csv', sep=';')
df_product[['id', 'name']].nunique()

id      496
name    487
dtype: int64

In [9]:
df_product = df_product[['id', 'name']]
df_product.dropna(inplace=True)
df_product.drop_duplicates(subset='name', inplace=True)

df_product['id_unique'] = pd.factorize(df_product['name'])[0]
df_product.set_index('id_unique', inplace=True)

df_product.head(2)

,id,name
id_unique,,
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг


In [10]:
df_productdealerkey = pd.read_csv(path_to_dir + 'marketing_productdealerkey.csv', sep=';')
df_productdealerkey.head(2)

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106


Данные готовы, можно приступить к предобработке текста.

## Предобработка текста

Были выявлены следующие недостатки в данных:

1. Разный регистр букв
2. Расхождение единиц объёма/веса
3. Слитное написание слов, отсутствие пробелов. Некоторые записи с полным отсутствием пробелов определяются как одно слово, что даёт `cosine_similarity = 0` до всех предложеных позиций.

Основная идея - привести все записи и слова к одному формату, чтобы увеличить количество совпадений по словам и снизить размерность векторного пространства для более точного определения `cosine_similarity`. На данный момент удалось снизить размерность вектора с 1222 слов до 1099.

In [11]:
def preprocess_text(text):
    text = text.lower() # нижний регистр

    # разделение слов
    text = re.compile(r'(?<=[а-яА-Я])(?=[A-Za-z])|(?<=[A-Za-z])(?=[а-яА-Я])').sub(" ", str(text))

    # преобразование мл в л
    text = re.sub(r'(\d+)\s*мл', lambda x: str(int(x.group(1)) / 1000) + ' ' + 'л', text)

    # очищение текст от знаков
    text = re.sub(r'[!#$%&\'()*+,./:;<=>?@[\]^_`{|}~—\"\\-]+', ' ', text)

    # разделение по определенным словам
    text = re.sub(r'(средство|мытья|для|чистящее|удаления|очистки|против|плесени|добавка|prosept)', r'\1 ', str(text))

    # удаление некоторых слов
    text = re.sub(r'\b(?:и|для|д|с|ф|п|ая|007|i)\b', '', text)

    return text

In [12]:
df_dealerprice.product_name, df_product.name = df_dealerprice.product_name.apply(preprocess_text), df_product.name.apply(preprocess_text)

Объединим два корпуса названий продуктов в один. Стоит отметить, что при использовании только `df_product.name` в качестве корпуса размерность вектора уменьшалась в два раза, однако метрика при этом становилась меньше на 0.004.

In [13]:
corpus = pd.concat([df_product.name, df_dealerprice.product_name]).drop_duplicates().values
corpus

array(['антисептик невымываемый prosept  ultra концентрат 1 10    1 л',
       'антигололед   32 prosept  готовый состав   12 кг',
       'герметик акриловый цвет сосна    0 6 л', ...,
       'средство    удаления  клейкой ленты  клея  наклеек 0 4л prosept  duty universal готовый состав',
       'отбеливатель   древесины prosept  eco 50 готовый состав 1 кг',
       'герметик акриловый межшовный   деревянных конструкций  цвет  орех  готовый состав 0 6 кг'],
      dtype=object)

Предобработка проведена. Её успешность оценивалась по размерности получаемого через `TfidfVectorizer()` вектора, о чём было сказано ранее, и по метрике, о чём будет сказано в дальнейшем, дабы не раздувать код дополнительными демонстрациями. Для `SentenceTransformer('LaBSE')` размерность вектора фиксирована и ровна `768`, что определяется выходным слоем `BERT`. Приступим к проверке гипотезы. Начнём в векторизации и создании функции предсказания.

## `tf-idf` и `cosine_similarity`

Векторизуем `df_product.name` и `df_dealerprice.product_name` с помощью `tf-idf`. Так же будем оценивать скорость выполнения каждой функциональной ячейки, так как время выполнения скрипта важный критерий для оценки всей работы.

In [14]:
%%time

vectorizer = TfidfVectorizer()
vectorizer_fited = vectorizer.fit(corpus)

vectors_targ_tf_idf = vectorizer_fited.transform(df_product.name).tocsc()
vectors_feat_tf_idf = vectorizer_fited.transform(df_dealerprice.product_name).tocsr()

CPU times: total: 46.9 ms
Wall time: 47.6 ms


Быстро. Параллельно посмотрим на размерность вектора.

In [15]:
len(vectors_targ_tf_idf.todense().tolist()[0])

1086

В функции предсказания выполняется итерирование по `vectors_feat`, рассчитывается `cosine_similarity` для каждой пары `vectors_feat[i], vectors_targ`. Функция возвращает лучшие `n` позиций вместе с соответствующими `cosine_similarity`. Для оптимизации времени расчёта матрицы векторов конвертированы в формат `Compressed Sparse`. <br>
В проекте `n` выставлен на `15`. Мы передаём в бэкенд больше мэтчей, чтобы при необходимости пользователь мог расширить диапазон сопоставления и при этом не ждать выполнения команд на расчёт. Мы можем позволить себе такую схему, так как мэтчи `всех со всеми` считаются довольно быстро.

In [16]:
def prediction(feat, vectors_feat, vectors_targ, n):
    
    pred_sim = cosine_similarity(vectors_feat, vectors_targ)
    top_n_indices = np.argpartition(pred_sim, -n)[:, -n:]
    top_n_values = pred_sim[np.arange(len(feat))[:,None], top_n_indices]
    sorted_indices = np.argsort(top_n_values, axis=1)[:, ::-1]

    pred = top_n_indices[np.arange(len(feat))[:,None], sorted_indices]
    pred_sim = np.around(top_n_values[np.arange(len(feat))[:,None], sorted_indices], 6)

    return pred, pred_sim

In [17]:
%%time

pred_tf_idf, pred_sim_tf_idf = prediction(df_dealerprice.product_name, vectors_feat_tf_idf, vectors_targ_tf_idf, 5)

CPU times: total: 15.6 ms
Wall time: 14.8 ms


**16** милисекунд - хороший результат, отличная скорость.

In [18]:
print(pred_tf_idf[0:5])

[[479 335 176 427 438]
 [ 63 174  89 263 262]
 [211 208 213 356 353]
 [458 247 276 144 192]
 [244 198 210 219 401]]


In [19]:
print(pred_sim_tf_idf[0:5])

[[0.713308 0.713308 0.636209 0.457675 0.457675]
 [0.760594 0.758475 0.675576 0.617725 0.617725]
 [0.92914  0.651239 0.651239 0.180895 0.159084]
 [0.701138 0.701138 0.660921 0.188367 0.165116]
 [0.910776 0.608842 0.466199 0.466199 0.39751 ]]


Функция работает и выдаёт корректный результат. Стоит отметить, что первая версия функции предсказания выполнялась за **~70** секунд и выдавала дублирующиеся значения айди названий от заказчика, то есть в топ-5 могло попасть всего 3 уникальных айди, что ухудшало не только метрику, но и выдачу для оператора в дальнейшем. <br>
Рассмотрим альтернативный метод получения векторов - `SentenceTransformer('LaBSE')`.

## `LaBSE` и `cosine_similarity`

Векторизуем `df_product.name` и `df_dealerprice.product_name` с помощью `SentenceTransformer('LaBSE')`. При этом будем использовать уменьшенную версию `LaBSE-en-ru`.

In [20]:
%%time

labse_model = SentenceTransformer('cointegrated/LaBSE-en-ru')

vectors_feat_LaBSE = labse_model.encode(df_dealerprice.product_name.tolist(), convert_to_tensor=True)
vectors_targ_LaBSE = labse_model.encode(df_product.name.tolist(), convert_to_tensor=True)

CPU times: total: 3min 37s
Wall time: 38.5 s


In [21]:
%%time

pred_LaBSE, pred_sim_LaBSE = prediction(df_dealerprice.product_name, vectors_feat_LaBSE, vectors_targ_LaBSE, 5)

CPU times: total: 125 ms
Wall time: 32 ms


In [22]:
print(pred_LaBSE[0:5])

[[479 335 306 257  62]
 [413 174 263  63 262]
 [211 208 270 213 479]
 [276 247 458 175 428]
 [244 267 413  60  46]]


In [23]:
print(pred_sim_LaBSE[0:5])

[[0.836639 0.807962 0.766614 0.755786 0.739337]
 [0.849344 0.809834 0.801016 0.797913 0.782371]
 [0.887787 0.738887 0.73254  0.730585 0.723519]
 [0.784702 0.722444 0.693823 0.576571 0.565438]
 [0.858922 0.735704 0.715354 0.706525 0.696894]]


Суммарное время расчёта в **40** секунд тоже является хорошим результатом, однако неизвестны параметры оборудования заказчика, на котором будет выполнятся наша программа, что может привести к значительному увеличению времени на выполнение кода. Так же необходимость загрузки тяжёлых библиотек является минусом этого векторизатора. Необходимо сравнить точность предсказанных мэтчей.

На данном этапе мы получили список предложенных нашей функцией `prediction` позиций заказчика для каждой позиции дилера. Однако данные индексы не соответствуют индексам во входных файлах, и для оценки полученных предсказаний необходимо сопоставить индексы настоящим `product_id` и `product_key`.

## Сопоставление нашего расчёта с `product_id` и `product_key`.

Напишем функцию для сопоставления айди.

In [24]:
def get_id_key(pred, df_product):
    result = []
    product_id = dict(df_product['id']) 

    for row in pred:
        new_row = [product_id.get(item, item) for item in row]
        result.append(new_row)
    return result

In [25]:
%%time

pred_id_key_tf_idf = get_id_key(pred_tf_idf, df_product)
pred_id_key_LaBSE = get_id_key(pred_LaBSE, df_product)

CPU times: total: 93.8 ms
Wall time: 11 ms


Создадим датафрейм с истинными `product_id` и `product_key`.

In [26]:
def result_to_df(pred, pred_sim, df_dealerprice):

    df_result = df_dealerprice['product_key']
    df_result = df_result.to_frame()
    df_result['product_id'] = pred
    df_result['pred_sim'] = pred_sim.tolist()
    
    return df_result

In [27]:
df_result_tf_idf = result_to_df(pred_id_key_tf_idf, pred_sim_tf_idf, df_dealerprice)
df_result_LaBSE = result_to_df(pred_id_key_LaBSE, pred_sim_LaBSE, df_dealerprice)

In [28]:
df_result_tf_idf.head(3)

,product_key,product_id,pred_sim
0,546227,"[12, 13, 15, 4, 5]","[0.713308, 0.713308, 0.636209, 0.457675, 0.457675]"
1,546408,"[470, 45, 434, 28, 29]","[0.760594, 0.758475, 0.675576, 0.617725, 0.617725]"
2,546234,"[18, 477, 478, 281, 282]","[0.92914, 0.651239, 0.651239, 0.180895, 0.159084]"


In [29]:
df_result_LaBSE.head(3)

,product_key,product_id,pred_sim
0,546227,"[12, 13, 69, 260, 488]","[0.8366389870643616, 0.8079620003700256, 0.7666140198707581, 0.7557860016822815, 0.7393370270729065]"
1,546408,"[38, 45, 28, 470, 29]","[0.84934401512146, 0.8098340034484863, 0.8010159730911255, 0.7979130148887634, 0.7823709845542908]"
2,546234,"[18, 477, 453, 478, 12]","[0.8877869844436646, 0.7388870120048523, 0.7325400114059448, 0.730584979057312, 0.7235190272331238]"



Проверим точность работы функции `predict`, используя полученные данные.

## Валидация решения

Валидацию наших решений проведём с помощью двух метрик - первая `metric_top_5` оценивает только попадание правильного названия заказчика в топ-5, вторая `mean_reciprocal_rank` учитывает позицию, на которой находится правильное предсказание.

In [30]:
def metric_top_5(actual, pred):
    count = 0
    for i in range(len(actual)):
        if actual[i] in pred[i]:
            count += 1
    
    return round(count / len(actual), 4)

Для проверки метрики воспользуемся датасетом `df_productdealerkey` с размеченными вручную мэтчами.

In [31]:
df_metric_tf_idf = df_result_tf_idf.merge(df_productdealerkey[['key', 'product_id']], left_on='product_key', right_on='key')
df_metric_tf_idf.head(3)

,product_key,product_id_x,pred_sim,key,product_id_y
0,546227,"[12, 13, 15, 4, 5]","[0.713308, 0.713308, 0.636209, 0.457675, 0.457675]",546227,12
1,546408,"[470, 45, 434, 28, 29]","[0.760594, 0.758475, 0.675576, 0.617725, 0.617725]",546408,38
2,546234,"[18, 477, 478, 281, 282]","[0.92914, 0.651239, 0.651239, 0.180895, 0.159084]",546234,18


In [32]:
df_metric_LaBSE = df_result_LaBSE.merge(df_productdealerkey[['key', 'product_id']], left_on='product_key', right_on='key')
df_metric_LaBSE.head(3)

,product_key,product_id_x,pred_sim,key,product_id_y
0,546227,"[12, 13, 69, 260, 488]","[0.8366389870643616, 0.8079620003700256, 0.7666140198707581, 0.7557860016822815, 0.7393370270729065]",546227,12
1,546408,"[38, 45, 28, 470, 29]","[0.84934401512146, 0.8098340034484863, 0.8010159730911255, 0.7979130148887634, 0.7823709845542908]",546408,38
2,546234,"[18, 477, 453, 478, 12]","[0.8877869844436646, 0.7388870120048523, 0.7325400114059448, 0.730584979057312, 0.7235190272331238]",546234,18


In [33]:
metric_top_5(df_metric_tf_idf.product_id_y, df_metric_tf_idf.product_id_x)

0.8652

In [34]:
metric_top_5(df_metric_LaBSE.product_id_y, df_metric_LaBSE.product_id_x)

0.8971

**0.8652** - 86% правильных названий заказчиков попали в топ-5 при применении `tf-idf`. Без блока с предобработкой текста данный параметр равнялся 76%. Для `SentenceTransformer('LaBSE')` результат немного лучше **0.8971** - 89%. <br>
Посмотрим, какой метод векторизации позволяет лучше ранжировать топ-5.

In [35]:
def mean_reciprocal_rank(true_id,
                         recommendations,
                         k=5):
    
    reciprocal_ranks = []
    
    for i, rec in enumerate(recommendations):
        recs = rec[:k]
        relevant = true_id[i]
        
        if np.isin(relevant, recs):
            rank = np.where(recs == relevant)[0][0] + 1
            reciprocal_ranks += [1 / rank]
            
        else:
            reciprocal_ranks += [0]
            
    return round(np.mean(reciprocal_ranks), 4)

In [36]:
mean_reciprocal_rank(df_metric_tf_idf.product_id_y, df_metric_tf_idf.product_id_x)

0.6375

In [37]:
mean_reciprocal_rank(df_metric_LaBSE.product_id_y, df_metric_LaBSE.product_id_x)

0.7888

По этой метрике `SentenceTransformer('LaBSE')` значительно лучше, чем `tf-idf`: **0.7888** против **0.6375**.

## Выбор векторайзера

Не смотря на то, что `SentenceTransformer('LaBSE')` показал заметно лучшие результаты по ранжированию, нежели `tf-idf`, но его интеграция в докер вызвала ряд проблем, таких как загрузка дополнительных тяжёлых библиотек `torch`, `nvidia` из интернета. Так же контейнер становится в два раза больше по объёму. В связи с сжатыми сроками проведения разработки было принято решение использовать более лёгкий, надёжный и быстрый `tf-idf`.

Сохраним предсказания `tf-idf` в файл с расширением `json`.

In [38]:
df_result_tf_idf.to_json(r'result.json', orient='columns')

Данный файл будет передаваться в бэкенд один раз при загрузке программы.<br>

Мы получили хорошие результаты, но пока всё равно не 100% мэтчей. Проверим позиции, по которым наша функция `predict` выдала ошибочные предсказания.

## Анализ ошибочных предсказаний

Создадим и запустим функцию `first_n_no_match` для визуализации ошибочных предсказаний.

In [39]:
def first_n_no_match(actual, pred, pred_sim, n, m):
    count = 0
    for i in range(n, m):
        if actual[i] not in pred[i]:
            print('i', i ,'actual' , actual[i], 'pred', pred[i], 'pred_sim', pred_sim[i])
            count += 1
    return count
    
print(first_n_no_match(df_metric_tf_idf.product_id_y, df_metric_tf_idf.product_id_x, df_metric_tf_idf.pred_sim, 1, 100))

i 1 actual 38 pred [470, 45, 434, 28, 29] pred_sim [0.760594, 0.758475, 0.675576, 0.617725, 0.617725]
i 17 actual 307 pred [259, 260, 261, 262, 306] pred_sim [0.484075, 0.484075, 0.449229, 0.43224, 0.398294]
i 19 actual 234 pred [496, 229, 228, 485, 232] pred_sim [0.385265, 0.26113, 0.26113, 0.261068, 0.244779]
i 25 actual 276 pred [271, 272, 273, 275, 274] pred_sim [0.790135, 0.740692, 0.711749, 0.701798, 0.672347]
i 29 actual 235 pred [496, 229, 228, 485, 232] pred_sim [0.385265, 0.26113, 0.26113, 0.261068, 0.244779]
i 42 actual 55 pred [53, 54, 50, 51, 52] pred_sim [0.438156, 0.438156, 0.423141, 0.423141, 0.42058]
i 43 actual 52 pred [54, 53, 50, 51, 57] pred_sim [0.373225, 0.373225, 0.360436, 0.360436, 0.358514]
i 60 actual 158 pred [155, 154, 157, 156, 159] pred_sim [0.429068, 0.429068, 0.408223, 0.408223, 0.399853]
i 63 actual 332 pred [334, 336, 335, 351, 350] pred_sim [0.593076, 0.593076, 0.593076, 0.589857, 0.589857]
i 64 actual 356 pred [334, 336, 335, 351, 350] pred_sim [0.5

Посмотрим на позицию `i 42 actual 55 pred [53, 54, 50, 51, 52] pred_sim [0.438065, 0.438065, 0.423054, 0.423054, 0.420493]`. Много предсказаний являются соседями. Также смущает одинаковое сходство для разных позиций.

Выведем позицию дилера, для которой получили неверное предсказание.

In [40]:
df_dealerprice[df_dealerprice.product_key == df_metric_tf_idf.iloc[42].product_key]

,product_key,product_name
42,716500,средство удаления ржавчины минеральных отложений 0 75л bath acid prosept цитрус 294 075


Выведем предсказанные позиции.

In [41]:
df_product[df_product.id.isin(df_metric_tf_idf.iloc[42].product_id_x)]

,id,name
id_unique,,
313,50,средство удаления ржавчины минеральных отложений щадящего действия bath acid концентрат 1 200 1 500 1 л
314,53,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 1 л
428,51,средство удаления ржавчины минеральных отложений щадящего действия bath acid концентрат 1 200 1 500 5 л
435,54,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 5 л
486,52,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 0 75 л


Выведем правильную позицию.

In [42]:
df_product[df_product.id == 55]

,id,name
id_unique,,
25,55,средство усиленного действия удаления ржавчины минеральных отложений bath acid ароматом цитрусаконцентрат 1 200 1 500 0 75 л


Это позиции заказчика для похожих товаров, которые отличаются объёмом и парой слов. А сходство идентично в тех позициях, где разница лишь в цифре объёма. Видимо, наш метод не очень хорошо различает цифры. Так же видно, что написание правильного названия сделано с ошибкой: в "цитрусаконцентрат" пропущен пробел. Скорее всего именно поэтому наш метод не смог сделать правильный мэтч. У нас есть блок предобработки текста, однако поиск всех подобных ошибок займёт много времени. В данном случае стоит передать заказчику информацию о том, что подобные ошибки ухудшают качество мэтчей, поэтому операторам стоит внимательнее относиться к занесению названий в базу данных.

Теперь посмотрим на позицию `i 63 actual 332 pred [334, 336, 335, 351, 350] pred_sim [0.593076, 0.593076, 0.593076, 0.589857, 0.589857]`

Так же выведем позицию дилера, для которой получили неверное предсказание.

In [43]:
df_dealerprice[df_dealerprice.product_key == df_metric_tf_idf.iloc[63].product_key]

,product_key,product_name
63,200671203,просепт bio lasur антисептик лессирующий защитно декоративный 2 7 л


Выведем предсказанные позиции.

In [44]:
df_product[df_product.id.isin(df_metric_tf_idf.iloc[63].product_id_x)]

,id,name
id_unique,,
22,351,антисептик лессирующий bio lasur орех 9 л
118,350,антисептик лессирующий bio lasur орех 2 7 л
120,336,антисептик лессирующий bio lasur бесцветный 9 л
146,335,антисептик лессирующий bio lasur бесцветный 2 7 л
151,334,антисептик лессирующий bio lasur бесцветный 0 9 л


Выведем правильную позицию.

In [45]:
df_product[df_product.id == 332]

,id,name
id_unique,,
150,332,антисептик лессирующий bio lasur белый люкс 2 7 л


В названии дилера отсутствует цвет, хотя у заказчика он указан. Кажется, что установить точный мэтч для такой позиции не представляется возможным, а предложенные и правильный варианты одинаково подходят под описание этого товара дилера. С таким типом ошибок в написаниях можно бороться только увеличением количества предлагаемых мэтчей.

И последний анализ позиции `i 93 actual 192 pred [8, 9, 10, 11, 67] pred_sim [0.622081, 0.622081, 0.484478, 0.484478, 0.351393]`

In [46]:
df_dealerprice[df_dealerprice.product_key == df_metric_tf_idf.iloc[93].product_key]

,product_key,product_name
93,100121642,просепт professional prof dz универсальное средство дезинфицирующим эффектом 5 л


In [47]:
df_product[df_product.id.isin(df_metric_tf_idf.iloc[93].product_id_x)]

,id,name
id_unique,,
269,10,универсальное средство на основе час дезинфицирующим эффектом un dz готовый состав 0 5 л
279,11,универсальное средство на основе час дезинфицирующим эффектом un dz готовый состав 5 л
315,67,средство удаления плесени дезинфицирующим эффектом bath fungi концентрат 1 50 1 100 0 5 л
439,9,универсальное моющее средство дезинфицирующим эффектом universal dz концентрат 1 10 1 120 5 л
443,8,универсальное моющее средство дезинфицирующим эффектом universal dz концентрат 1 10 1 120 1 л


In [48]:
df_product[df_product.id == 192]

,id,name
id_unique,,
133,192,дезинфицирующее средство проф дз prof dz готовый состав 5 л


Тут видно, что описание у дилеров действительно не соответствует описанию заказчика: есть лишние слова "универсальное" и "эффектом", но при этом остуствует "готовый состав". Настолько большое отличие при наличии подходящих кондидатов наш метод уловить не может.

### Выводы и планы

Выбранная базовая гипотеза подтвердилась, предложенный для её реализации метод дал отличные результаты. При этом в используемых названиях от заказчика и дилеров всё еще есть недостатки, ликвидируя которые можно дополнительно поднять точность мэтчей.

В ходе выполнения проекта релизована предобработка данных. Разработан метод сопоставления топ `n` названий заказчиков названиям дилеров. Получены отличная скорость выполнения кода и отличное значение метрики.